In [51]:
!pip install azureml
!pip install azureml.core
!pip install azureml.widgets
!pip install azureml.pipeline

  Using cached azureml_core-1.27.0-py3-none-any.whl (2.2 MB)
ERROR: azureml-defaults 1.21.0 has requirement azureml-core~=1.21.0, but you'll have azureml-core 1.27.0 which is incompatible.
ERROR: azureml-defaults 1.21.0 has requirement azureml-dataset-runtime[fuse]~=1.21.0, but you'll have azureml-dataset-runtime 1.27.0 which is incompatible.
ERROR: azureml-defaults 1.21.0 has requirement flask==1.0.3, but you'll have flask 1.1.2 which is incompatible.


  Using cached azureml_widgets-1.27.0-py3-none-any.whl (14.1 MB)


  Using cached azureml_pipeline-1.27.0-py3-none-any.whl (3.7 kB)


In [52]:
import os
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset, Run
from azureml.widgets import RunDetails
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core.pipeline_output_dataset import PipelineOutputFileDataset

In [53]:
ws = Workspace.from_config()

In [54]:
keyvault = ws.get_default_keyvault()

In [55]:
default_datastore = ws.get_default_datastore()

### Define all trials datasets

In [56]:
dataset_who_vaccine_landscape_covid19_name = 'who-vaccine-landscape-covid19'
dataset_who_vaccine_landscape_covid19 = Dataset.get_by_name(workspace=ws, name=dataset_who_vaccine_landscape_covid19_name)

In [57]:
dataset_trials_landscape_name = 'trials-landscape'
dataset_trials_landscape = Dataset.get_by_name(workspace=ws, name=dataset_trials_landscape_name)

In [58]:
dataset_clinicaltrialsgov_name = 'clinicaltrialsgov'
dataset_clinicaltrialsgov = Dataset.get_by_name(workspace=ws, name=dataset_clinicaltrialsgov_name)

In [59]:
dataset_aact_trials_name = 'aact-trials'
dataset_aact_trials = Dataset.get_by_name(workspace=ws, name=dataset_aact_trials_name)

### Create CPU Compute

In [60]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
aml_cpu_compute_cluster_name = 'cpucluster-d4-v3'
vm_size = 'STANDARD_D4_V3'

# Verify that cluster does not exist already
try:
    aml_cpu_compute = ComputeTarget(workspace=ws, name=aml_cpu_compute_cluster_name)
    print('Found existing cluster {}'.format(aml_cpu_compute_cluster_name))
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                           max_nodes=4)
    aml_cpu_compute = ComputeTarget.create(ws, aml_cpu_compute_cluster_name, compute_config)

aml_cpu_compute.wait_for_completion(show_output=True)

Found existing cluster cpucluster-d4-v3
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Define Runtime Configuration

In [61]:
# https://docs.microsoft.com/en-us/azure/container-registry/container-registry-auth-service-principal

from azureml.core.runconfig import RunConfiguration
aml_run_config = RunConfiguration()

from azureml.core.environment import Environment
# Create the environment
rp_env = Environment(name='rp')
# Enable Docker and reference an image
rp_env.docker.enabled = True
rp_env.python.user_managed_dependencies=True

rp_env.docker.base_image_registry.address = "acieurfrcaassacr.azurecr.io"
rp_env.docker.base_image_registry.username = keyvault.get_secret(name='acieurfrcaassacr-admin-user')
rp_env.docker.base_image_registry.password = keyvault.get_secret(name='acieurfrcaassacr-admin-pwd')
rp_env.docker.base_image = "acieurfrcaassacr.azurecr.io/azureml-env-base-research-platform:latest"

aml_run_config.environment = rp_env
rp_env.register(workspace=ws)

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "acieurfrcaassacr.azurecr.io/azureml-env-base-research-platform:latest",
        "baseImageRegistry": {
            "address": "acieurfrcaassacr.azurecr.io",
            "password": "AzureMlSecret=Env_796e80c7e5bf8a48cb603f229eeec578ec72443dbfe37710cc80de67228c6713_1#EnvironmentDefinition#ContainerRegistry#Password",
            "registryIdentity": null,
            "username": "acieurfrcaassacr"
        },
        "enabled": true,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "rp",
    

## Specify script source folder(s)

In [62]:
dataprep_script_folder_who_vaccine_landscape = '../who-vaccine-landscape/'
print('Source directory for WHO vaccine landscape data preparation is {}.'.format(os.path.realpath(dataprep_script_folder_who_vaccine_landscape)))

dataprep_script_folder_trials_landscape = '../trials-landscape/'
print('\nSource directory for trials landscape data preparation is {}.'.format(os.path.realpath(dataprep_script_folder_trials_landscape)))

dataprep_script_folder_clinicaltrialsgov = '../clinicaltrialsgov/'
print('\nSource directory for clinicaltrialsgov data preparation is {}.'.format(os.path.realpath(dataprep_script_folder_clinicaltrialsgov)))

dataprep_script_folder_aact_trials = '../aact-trials/'
print('\nSource directory for aact trials data preparation is {}.'.format(os.path.realpath(dataprep_script_folder_aact_trials)))

Source directory for WHO vaccine landscape data preparation is /media/shared/My Documents/My Developments/research-platform/code/dataprep/clinical-trials/who-vaccine-landscape.

Source directory for trials landscape data preparation is /media/shared/My Documents/My Developments/research-platform/code/dataprep/clinical-trials/trials-landscape.

Source directory for clinicaltrialsgov data preparation is /media/shared/My Documents/My Developments/research-platform/code/dataprep/clinical-trials/clinicaltrialsgov.

Source directory for aact trials data preparation is /media/shared/My Documents/My Developments/research-platform/code/dataprep/clinical-trials/aact-trials.


### Step - Extract WHO vaccine landscape from .pdf/.xlsx to .json

In [63]:
trials_out = PipelineOutputFileDataset(PipelineData(name="dataset_trials_landscape",datastore=default_datastore))

step_dataprep_who_extract = PythonScriptStep(
    name='Extract WHO to trials JSON',
    script_name='step_dataprep_extract.py', 
    arguments=['--input', dataset_who_vaccine_landscape_covid19.as_named_input('raw_who_data').as_mount(),
               '--output_dataset', 'trials-landscape',
               '--disease', 'COVID19'],
    outputs=[trials_out],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder_who_vaccine_landscape,
    allow_reuse=True
)

print('Step Dataprep Extract WHO created')

Step Dataprep Extract WHO created


### Step - Update database from trials corpus (JSON)

In [64]:
# Database update
step_database_update_trials_landscape = PythonScriptStep(
    name='Trails JSON to Database Update',
    script_name='step_database_update.py', 
    arguments=['--input', dataset_trials_landscape.as_named_input('raw_json_data').as_mount()],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder_trials_landscape,
    allow_reuse=True
)

print('Step Database Update Trails Landscape created')

Step Database Update Trails Landscape created


In [65]:
step_database_update_trials_landscape.run_after(step_dataprep_who_extract)

### Step - Update database from ClinicalTrials.Gov trials extract

In [66]:
# Database update
step_database_update_clinicaltrialsgov = PythonScriptStep(
    name='ClinicalTrialsGov to Database Update',
    script_name='step_database_update.py', 
    arguments=['--input', dataset_clinicaltrialsgov.as_named_input('raw_csv_data').as_mount()],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder_clinicaltrialsgov,
    allow_reuse=True
)

print('Step Database Update ClinicalTrialsGov created')

Step Database Update ClinicalTrialsGov created


In [67]:
step_database_update_clinicaltrialsgov.run_after(step_database_update_trials_landscape)

### Step - Update database from AACT extract (aggregate analysis of Clinical Trials)

In [68]:
# Load step creation
step_database_update_aact = PythonScriptStep(
    name='AACT to Database Update',
    script_name='step_database_update.py', 
    arguments=['--input', dataset_aact_trials.as_named_input('raw_data').as_mount(),
               '--extract_terms', 'condition_COVID-19_coronavirus'],
    compute_target=aml_cpu_compute,
    runconfig=aml_run_config,
    source_directory=dataprep_script_folder_aact_trials,
    allow_reuse=True
)

print('Step Database Update created')

Step Database Update created


In [69]:
step_database_update_aact.run_after(step_database_update_clinicaltrialsgov)

### Create Step List

In [70]:
steps = [step_dataprep_who_extract,
         step_database_update_trials_landscape,
         step_database_update_clinicaltrialsgov,
         step_database_update_aact]
print("Step lists created")

Step lists created


### Create and validate pipeline

In [71]:
aml_pipeline = Pipeline(workspace=ws, steps=steps)
print ("Pipeline is built")

Pipeline is built


In [72]:
aml_pipeline.validate()
print("Pipeline validation complete")

Pipeline validation complete


### Submit experiment for execution

In [73]:
# Note regenerate output, decides whether to execute the module to regenerate output at running time.
# It's by default unselected, which means if the module has been executed with the same parameters previously, 
# the system will reuse the output from last run to reduce run time.
aml_pipeline_run = Experiment(ws, 'pipeline-clinical-trials').submit(aml_pipeline, regenerate_outputs=True)
print("Experiment pipeline clinical-trials is submitted for execution")

Created step Extract WHO to trials JSON [f43cb6a6][39661c45-fb21-457a-a821-40deea338d6d], (This step will run and generate new outputs)
Created step Trails JSON to Database Update [80d8996c][edfe669d-9f87-44b9-ba09-492be018a8a2], (This step will run and generate new outputs)Created step ClinicalTrialsGov to Database Update [0d2dc56d][5fa6190f-4025-4733-ad3a-41d462ada869], (This step will run and generate new outputs)
Created step AACT to Database Update [025cc5d9][858d6030-a475-4704-902c-ac1fc96a6851], (This step will run and generate new outputs)

Submitted PipelineRun beb84267-0d6f-40f6-913c-1a8e714d172f
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/beb84267-0d6f-40f6-913c-1a8e714d172f?wsid=/subscriptions/19518d47-0c8b-4829-a602-c5ced78deb3f/resourcegroups/aci-eur-frc-aa-ss-rg/workspaces/aci-eur-frc-aa-ss-mlw&tid=ee1a033c-2d44-4728-891d-4ef598619020
Experiment pipeline clinical-trials is submitted for execution


In [49]:
RunDetails(aml_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [50]:
step_runs = aml_pipeline_run.get_children()
for step_run in step_runs:
    status = step_run.get_status()
    print('Script:', step_run.name, 'status:', status)
    
    # Change this if you want to see details even if the Step has succeeded.
    if status == "Failed":
        joblog = step_run.get_job_log()
        print('job log:', joblog)